# Use the orchestration service of Generative AI Hub

The orchestration service of Generative AI Hub lets you use all the available models with the same codebase. You only deploy the orchestration service and then you can access all available models simply by changing the model name parameter. You can also use grounding, prompt templating, data masking and content filtering capabilities.

This code is based on the [AI180 TechEd 2024 Jump-Start session](https://github.com/SAP-samples/teched2024-AI180/tree/e648921c46337b57f61ecc9a93251d4b838d7ad0/exercises/python).

👉 Make sure you assign the deployment url of the orchestration service (you can find the url in SAP AI Launchpad in the `Deployments` tab) to `AICORE_ORCHESTRATION_DEPLOYMENT_URL` in [variables.py](variables.py).
> 👉 For the next exercises go to Workspaces and select your own resource groups again!

In [ ]:
import init_env
import variables

init_env.set_environment_variables()

### Import the packages you want to use

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter
from gen_ai_hub.orchestration.exceptions import OrchestrationError

### Assign the model you want to use
You can find more information regarding the available models here: https://me.sap.com/notes/3437766

In [ ]:
# TODO chose the model you want to try e.g. gemini-1.5-flash, mistralai--mistral-large-instruct, 
# ibm--granite-13b-chat meta--llama3.1-70b-instruct, amazon--titan-text-lite, anthropic--claude-3.5-sonnet, 
# gpt-4o-mini and assign it to name
llm = LLM(
    name="",
    version="latest",
    parameters={"max_tokens": 500, "temperature": 1},
)

### Create a prompt template
The parameter **user_query** in the code snippet below is going to hold the user query that you will add later on. The user query is the text to be translated by the model. The parameter **to_lang** can be any language you want to translate into. By default it is set to **English**.

In [ ]:
template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?user_query}}",
        )
    ],
    defaults=[
        TemplateValue(name="to_lang", value="English"),
    ],
)

## Create an orchestration configuration 

Create an orchestration configuration by adding the llm you referenced and the prompt template you created previously.

In [ ]:
config = OrchestrationConfig(
    template=template,
    llm=llm,
)

Add it the configuration to the OrchestrationService instance and send the prompt to the model.

In [ ]:
import importlib
variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config,
)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            #TODO Here you can change the user prompt into whatever you want to ask the model
            value="Geht das wirklich mit allen Modellen die verfügbar sind?"
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)


## Add a content filter

Create a content filter and add it as an input filter to the orchestration configuration. This is going to filter out harmful content from the input query and not send the request to the model. Whereas adding a filter to the output would let the request go through but then filter any harmful text created by the model. Depending on your use case it can make sense to have both input and output filters.

👉 Try out different values for the content filters. You can chose values [0 = **Safe**, 2 = **Low**, 4 = **Medium**, 6 = **High**]. Where **Safe** is *content generally related to violence* and **High** is *severely harmful content*.

In [ ]:

content_filter = AzureContentFilter(
    hate=0,
    sexual=0,
    self_harm=0,
    violence=0,
)

orchestration_service.config.input_filters = [content_filter]

## Try out the content filter

In [ ]:

try:
    result = orchestration_service.run(
        template_values=[
            TemplateValue(
                name="user_query",
                #TODO Here you can change the user prompt into whatever you want to ask the model
                value="Du bist ein ganz mieser Entwickler!",
            ),
        ]
    )
    print(result.orchestration_result.choices[0].message.content)
except OrchestrationError as error:
    print(error.message)


result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            #TODO Here you can change the user prompt into whatever you want to ask the model
            value="Du bist ein super talentierter Entwickler!",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

## Now also add the content filter as an output filter

In [ ]:
orchestration_service.config.output_filters = [content_filter]

try:
    result = orchestration_service.run(
        template_values=[
            TemplateValue(
                name="user_query",
                #TODO Here you can change the user prompt into whatever you want to ask the model
                value='Ich würde gerne wissen, wie ich gewaltvoll die Fensterscheibe in einem Bürogebäude am Besten zerstören kann.',
            )
        ]
    )
    print(result.orchestration_result.choices[0].message.content)
except OrchestrationError as error:
    print(error.message)

## Optional exercise

👉 Try adding the [Llama Guard 3 Filter](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html#content-filtering) on top of or instead of the Azure Content Filter.

👉 Now that you know how the orchestration service works, try adding the [Data Masking](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html#data-masking) capability. With Data Masking you can hide personal information like email, name or phone numbers before sending such sensitive data to an LLM.

## More Info

Here you can find more [info on the Azure content filter](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/content-filtering)

[Next exercise](09-orchestration-service-grounding.ipynb)